In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
#TensorFlow Version

def dynamic_k_matching(cost, pair_wise_ious, gt_classes, num_gt, fg_mask):
  # matching_matrix = tf.Variable(tf.zeros(cost.shape[0] * cost.shape[1]))
  matching_matrix = tf.zeros_like(cost, dtype=tf.uint8)
  print(f"matching matrix: {matching_matrix}")


  with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
  flattened_idx = []
  ious_in_boxes_matrix = pair_wise_ious
  n_candidate_k = min(10, ious_in_boxes_matrix.shape[1])
  print(f"n_candidate_k: {n_candidate_k}, ious_in_boxes_matrix.shape[1]: {ious_in_boxes_matrix.shape[1]}")
  
  #tf.math.top_k() causes issues/error when called under session. For some reason it is not a tf.function
  topk_ious, _ = tf.math.top_k(ious_in_boxes_matrix, k=n_candidate_k)
  print(f"topk_ious: {topk_ious}")

  #changed to casting after reduce_sum (as requested from wenxin)
  dynamic_ks = tf.math.maximum(tf.cast(tf.reduce_sum(topk_ious, axis=1), tf.int32), [1])
  
  print(f"dynamic_ks: {dynamic_ks}")
  
  for gt_idx in range(num_gt):
      num_top = dynamic_ks[gt_idx]
      _, pos_idx = tf.math.top_k(
          -cost[gt_idx], k=num_top, sorted=False)
      
      print(f"pos_idx: {pos_idx}")

      # indices and update necessary for tf.tensor_scatter_nd_update but not sure if this is optimized for tpus
      indices = []
      updates = []
      for idx in pos_idx:
        indices.append([gt_idx, idx])
        updates.append(1)

      matching_matrix = tf.tensor_scatter_nd_update(matching_matrix, indices, updates)
      # matching_matrix[gt_idx][idx] = 1
      
  print(f"matching_matrix: {matching_matrix}")

  del topk_ious, dynamic_ks, pos_idx

  anchor_matching_gt = tf.math.reduce_sum(matching_matrix, 0)
  print(f"anchor_matching_gt = {anchor_matching_gt}")
  

  #### NO longer needed
  # maybe change name of var later
  ### this block is equivalent to (anchor_matching_gt > 1).sum() in pytorch
  # num_anchor_matched = 0
  # for anchor in anchor_matching_gt:
  #   if anchor > 1:
  #     num_anchor_matched += 1
  ###

  print(f"val for if statement: {tf.math.reduce_sum(tf.cast((anchor_matching_gt > 1), tf.int32))}")
  if tf.math.reduce_sum(tf.cast((anchor_matching_gt > 1), tf.int32)) > 0:
    mask = anchor_matching_gt > 1
    cost_after_mask = tf.boolean_mask(cost, mask, axis=1)
    cost_argmin = tf.math.argmin(cost_after_mask)
    print(f"cost_argmin: {cost_argmin}")
    print(f"anchor_matching_gt > 1: {anchor_matching_gt > 1}")

    #getting indices for matching_matrix[:, anchor_matching_gt > 1]
    #getting for every row
    mask = tf.reshape(mask, [1, len(mask)])
    num_repeat = tf.constant([anchor_matching_gt.shape[0], 1], tf.int32)
    tiled_mask = tf.tile(mask, num_repeat)
    indices = tf.where(tiled_mask)
    updates = []
    for ind in indices:
      updates.append(0)

    matching_matrix = tf.tensor_scatter_nd_update(matching_matrix, indices, updates)
    print(f"matching_matrix: {matching_matrix}")
    
    indices = []
    updates = []
    for ind, mask_val in enumerate(mask[0]):
      if mask_val:
        cost_ind = cost_argmin[ind]
        index = [cost_ind, ind]
        indices.append(index)
        updates.append(1)

    matching_matrix = tf.tensor_scatter_nd_update(matching_matrix, indices, updates)
    print(f"matching_matrix: {matching_matrix}")
  
  ##############
  # fg_mask_inboxes = matching_matrix.sum(0) > 0
  fg_mask_inboxes = tf.math.reduce_sum(matching_matrix, axis=0) > 0

  # num_fg = fg_mask_inboxes.sum().item()
  num_fg = tf.math.reduce_sum( tf.cast(fg_mask_inboxes, tf.float32) )
  print(f"fg_mask_inboxes: {fg_mask_inboxes}, num_fg: {num_fg}\n")

  # fg_mask[fg_mask.clone()] = fg_mask_inboxes
  fg_mask = tf.boolean_mask(fg_mask_inboxes, fg_mask)
  ## MAKE SURE IT WORKS
  print(f"fg_mask: {fg_mask}\n")

  # matched_gt_inds = matching_matrix[:, fg_mask_inboxes].argmax(0)
  matched_gt_inds = tf.math.argmax(tf.boolean_mask(matching_matrix, fg_mask_inboxes, axis=1))
  print(f"matched_gt_inds: {matched_gt_inds}")

  # gt_matched_classes = gt_classes[matched_gt_inds]
  gt_matched_classes = tf.gather(gt_classes, matched_gt_inds)
  print(f"gt_matched_classes: {gt_matched_classes}")


  pred_ious_this_matching_temp1 = tf.math.multiply(tf.cast(matching_matrix, tf.float32), pair_wise_ious)
  pred_ious_this_matching_temp2 = tf.math.reduce_sum(pred_ious_this_matching_temp1, axis=0) 
  pred_ious_this_matching = tf.boolean_mask(pred_ious_this_matching_temp2, fg_mask_inboxes)
  print(f"pred_ious_this_matching: {pred_ious_this_matching}")

  return num_fg, gt_matched_classes, pred_ious_this_matching, matched_gt_inds

  

In [ ]:
# cost = tf.ones([8, 8])
cost = tf.convert_to_tensor([[0.1081, 0.4863, 0.1679, 0.3832, 0.0141, 0.6604, 0.1300, 0.1762],
        [0.1400, 0.1980, 0.2087, 0.3757, 0.5950, 0.6953, 0.8711, 0.6065],
        [0.0214, 0.7250, 0.5053, 0.1129, 0.5049, 0.9175, 0.8503, 0.7133],
        [0.0845, 0.9322, 0.1333, 0.6464, 0.6768, 0.0621, 0.4914, 0.3330],
        [0.7293, 0.9651, 0.5887, 0.0382, 0.1010, 0.0868, 0.0012, 0.8953],
        [0.1871, 0.5527, 0.1132, 0.9443, 0.3355, 0.4003, 0.1088, 0.6429],
        [0.1602, 0.0666, 0.7899, 0.2332, 0.7720, 0.1733, 0.2885, 0.7663],
        [0.8238, 0.1027, 0.5462, 0.9049, 0.0366, 0.5497, 0.1775, 0.4196]])
# pair_wise_ious = tf.ones([8, 2880])
pair_wise_ious = tf.convert_to_tensor([[0.6325, 0.5803, 0.5652, 0.5558, 0.5212, 0.4099, 0.3997, 0.3588],
        [0.7447, 0.6987, 0.5659, 0.4033, 0.3787, 0.2296, 0.1168, 0.0662],
        [0.8883, 0.6677, 0.6483, 0.6067, 0.3392, 0.1518, 0.1076, 0.0939],
        [0.9348, 0.7530, 0.6374, 0.5903, 0.3919, 0.2885, 0.2733, 0.0110],
        [0.9011, 0.6985, 0.6685, 0.5251, 0.2050, 0.1975, 0.0348, 0.0318],
        [0.9383, 0.9098, 0.8053, 0.7286, 0.7005, 0.2340, 0.1698, 0.0046],
        [0.9737, 0.8708, 0.6161, 0.4645, 0.4458, 0.3038, 0.0854, 0.0497],
        [0.7961, 0.5860, 0.5669, 0.5319, 0.4365, 0.3040, 0.1664, 0.1067]])
gt_classes = tf.convert_to_tensor([2.0, 5.0, 63., 50., 9., 10., 70., 15.])
num_gt = 8
fg_mask = tf.cast(tf.ones(8), tf.bool)

dynamic_k_matching(cost, pair_wise_ious, gt_classes, num_gt, fg_mask)

matching matrix: [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
n_candidate_k: 8, ious_in_boxes_matrix.shape[1]: 8
topk_ious: [[0.6325 0.5803 0.5652 0.5558 0.5212 0.4099 0.3997 0.3588]
 [0.7447 0.6987 0.5659 0.4033 0.3787 0.2296 0.1168 0.0662]
 [0.8883 0.6677 0.6483 0.6067 0.3392 0.1518 0.1076 0.0939]
 [0.9348 0.753  0.6374 0.5903 0.3919 0.2885 0.2733 0.011 ]
 [0.9011 0.6985 0.6685 0.5251 0.205  0.1975 0.0348 0.0318]
 [0.9383 0.9098 0.8053 0.7286 0.7005 0.234  0.1698 0.0046]
 [0.9737 0.8708 0.6161 0.4645 0.4458 0.3038 0.0854 0.0497]
 [0.7961 0.586  0.5669 0.5319 0.4365 0.304  0.1664 0.1067]]
dynamic_ks: [4 3 3 3 3 4 3 3]
pos_idx: [2 0 6 4]
pos_idx: [2 1 0]
pos_idx: [4 3 0]
pos_idx: [2 0 5]
pos_idx: [5 3 6]
pos_idx: [4 0 2 6]
pos_idx: [5 1 0]
pos_idx: [6 1 4]
matching_matrix: [[1 0 1 0 1 0 1 0]
 [1 1 1 0 0 0 0 0]
 [1 0 0 1 1 0 0 0]
 [1 0 1 0 0 1 0 0]
 [0 0 0 1 0 1 1 0]
 [1 0 1 0 1 0

(<tf.Tensor: shape=(), dtype=float32, numpy=7.0>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=array([63., 70., 10.,  9.,  2., 50.,  9.], dtype=float32)>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([0.8883, 0.8708, 0.8053, 0.5251, 0.5212, 0.2885, 0.0348],
       dtype=float32)>,
 <tf.Tensor: shape=(7,), dtype=int64, numpy=array([2, 6, 5, 4, 0, 3, 4])>)

In [ ]:
test = tf.constant([1, 2, 3, 5, 0, -1, 2])
test2 = tf.Variable([1, 2, 3, 5, 0, -1, 2])
print(tf.math.reduce_sum(tf.cast((test > 1), tf.int32)))
print(test2 > 1)

# cost[:, tf.cast(test > 1, tf.int32)]

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([False  True  True  True False False  True], shape=(7,), dtype=bool)


In [ ]:
anchor_matching_gt = tf.constant([6, 3, 4, 2, 4, 3, 4, 0])
matching_matrix = tf.constant([[1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 1, 0, 0, 0],
        [1, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 1, 0],
        [1, 0, 1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 1, 0]])

mask = anchor_matching_gt > 1
tf.boolean_mask(cost, mask, axis=1)
print(f"mask: {mask}")
# masked_matching_matrix = tf.boolean_mask(matching_matrix, mask,axis=1)
# print(masked_matching_matrix)
print(tf.shape(mask))
mask = tf.reshape(mask, [1, len(mask)])
print(mask)
num_repeat = tf.constant([4, 1], tf.int32)
print(tf.tile(mask, num_repeat))
print(tf.where(mask))
# matching_matrix *= 0
# for row in range(matching_matrix.shape[0]):
  # matching_matrix[row, :] *= 3
  # print(matching_matrix[row, :])

mask: [ True  True  True  True  True  True  True False]
tf.Tensor([8], shape=(1,), dtype=int32)
tf.Tensor([[ True  True  True  True  True  True  True False]], shape=(1, 8), dtype=bool)
tf.Tensor(
[[ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]], shape=(4, 8), dtype=bool)
tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]], shape=(7, 2), dtype=int64)
